### Hadoop and Hive

Hadoop was first released by Apache in 2011 as Version 1.0.0, which only contained HDFS and MapReduce. Hadoop was designed as both a computing (MapReduce) and storage (HDFS) platform from the very beginning. With the increasing need for big data analysis, Hadoop attracts lots of other software to resolve big data questions and merges into a Hadoop-centric big data ecosystem. 

Hive is a standard for SQL queries over petabytes of data in Hadoop. It provides SQL-like access to data in HDFS, enabling Hadoop to be used as a data warehouse. The Hive Query Language (HQL) has similar semantics and functions as standard SQL in the relational database, so that experienced database analysts can easily get their hands on it. Hive's query language can run on different computing engines, such as MapReduce, Tez, and Spark.

Hive's metadata structure provides a high-level, table-like structure on top of HDFS. It supports three main data structures, tables, partitions, and buckets. The tables correspond to HDFS directories and can be divided into partitions, where data files can be divided into buckets. Hive's metadata structure is usually the Schema of the Schema-on-Read concept on Hadoop, which means you do not have to define the schema in Hive before you store data in HDFS. Applying Hive metadata after storing data brings more flexibility and efficiency to your data work. The popularity of Hive's metadata makes it the de facto way to describe big data and is used by many tools in the big data ecosystem. 

Here are some highlights of Hive that we can keep in mind moving forward:

    Hive provides a simple and optimized query model with less coding than MapReduce
    HQL and SQL have a similar syntax
    Hive's query response time is typically much faster than others on the same volume of big datasets
    Hive supports running on different computing frameworks
    Hive supports ad hoc querying data on HDFS and HBase
    Hive supports user-defined java/scala functions, scripts, and procedure languages to extend its functionality
    Matured JDBC and ODBC drivers allow many applications to pull Hive data for seamless reporting
    Hive allows users to read data in arbitrary formats, using SerDes and Input/Output formats
    Hive is a stable and reliable batch-processing tool, which is production-ready for a long time
    Hive has a well-defined architecture for metadata management, authentication, and query optimizations
    There is a big community of practitioners and developers working on and using Hive


Cloudera Data Warehouse (CDW) Data Service is a containerized application for creating highly performant, independent, self-service data warehouses in the cloud which can be scaled dynamically and upgraded independently. Learn more about the service architecture, and how CDW enables data practitioners and IT administrators to achieve their goals.

Cloudera Machine Learning now offers Snippet to connect to Data Sources available within the CDP Environment. Administrators can configure custom Spark, Hive or Impala Virtual Warehouse data connections manually or they can use CML’s features to autodetect and configure all connections from the same CDP Environment. Data Scientists can then access the preconfigured Data Connections from their ML Projects.

In [2]:
import cml.data_v1 as cmldata

CONNECTION_NAME = "default-hive-aws"
conn = cmldata.get_connection(CONNECTION_NAME)

## Sample Usage to get pandas data frame
EXAMPLE_SQL_QUERY = "show databases"
dataframe = conn.get_pandas_dataframe(EXAMPLE_SQL_QUERY)
print(dataframe)

## Other Usage Notes:

## Alternate Sample Usage to provide different credentials as optional parameters
#conn = cmldata.get_connection(
#    CONNECTION_NAME, {"USERNAME": "someuser", "PASSWORD": "somepassword"}
#)

## Alternate Sample Usage to get DB API Connection interface
#db_conn = conn.get_base_connection()

## Alternate Sample Usage to get DB API Cursor interface
#db_cursor = conn.get_cursor()
#db_cursor.execute(EXAMPLE_SQL_QUERY)
#for row in db_cursor:
#  print(row)

            database_name
0             01_car_data
1               01_car_dw
2          adash_car_data
3                 airline
4              airline_dw
..                    ...
143  user_test_3_car_data
144                vademo
145         worldwidebank
146           yanliu_test
147     yingchen_car_data

[148 rows x 1 columns]


### Hive Tables

The concept of a table in Hive is very similar to the table in the relational database. Each table maps to a directory, which is under /user/hive/warehouse by default in HDFS. For example, /user/hive/warehouse/employee is created for the employee table. All the data in the table is stored in this hive user-manageable directory (full permission). This kind of table is called an internal, or managed, table. When data is already stored in HDFS, an external table can be created to describe the data. It is called external because the data in the external table is specified in the LOCATION property rather than the default warehouse directory. When keeping data in the internal tables, the table fully manages the data in it. When an internal table is dropped, its data is deleted together. However, when an external table is dropped, the data is not deleted. It is quite common to use external tables for source read-only data or sharing the processed data to data consumers giving customized HDFS locations. On the other hand, the internal table is often used as an intermediate table during data processing, since it is quite powerful and flexible when supported by HQL.

In [9]:
SQL_QUERY = "SHOW TABLES '*customer*'"

In [10]:
dataframe = conn.get_pandas_dataframe(SQL_QUERY)
print(dataframe)

                                 tab_name
0                        01_customer_data
1        ca_customer_issues_per_month_csv
2                  customer_daily_profile
3                           customer_data
4                           customer_demo
5                 customer_profiles_table
6                           customer_temp
7        customer_terminal_profiles_table
8   customer_terminal_profiles_table_test
9                            customerdata
10                         customerdata1g
11                      customerdata_temp
12                          customerdatag
13                        customerrecords
14                              customers
15                             customers2
16                            customers_2
17                         customers_ofer
18                          customers_srm
19                   historical_customers
20                olist_customers_dataset
21                      ww_customers_data


In [11]:
SQL_QUERY = "SHOW TBLPROPERTIES customers"

In [12]:
dataframe = conn.get_pandas_dataframe(SQL_QUERY)
print(dataframe)

               prpt_name                                         prpt_value
0                comment                                                   
1        kudu.cluster_id                   f3b70651dc5f44eb9e17aef582e0689f
2  kudu.master_addresses  real-time-datamart-master10.go01-dem.ylcu-atmi...
3          kudu.table_id                   4e0f48639b8d4e4096f6d446129004a3
4        kudu.table_name                                  default.customers
5        storage_handler     org.apache.hadoop.hive.kudu.KuduStorageHandler
6  transient_lastDdlTime                                         1658862739


In [13]:
SQL_QUERY = "SHOW CREATE TABLE customers"

In [14]:
dataframe = conn.get_pandas_dataframe(SQL_QUERY)
print(dataframe)

                                       createtab_stmt
0                           CREATE TABLE `customers`(
1                        `acc_id` string COMMENT '', 
2                        `f_name` string COMMENT '', 
3                        `l_name` string COMMENT '', 
4                         `email` string COMMENT '', 
5                        `gender` string COMMENT '', 
6                         `phone` string COMMENT '', 
7                           `card` string COMMENT '')
8                                          COMMENT ''
9                                   ROW FORMAT SERDE 
10                                                '' 
11                                         STORED BY 
12    'org.apache.hadoop.hive.kudu.KuduStorageHand...
13                                                   
14                                           LOCATION
15    's3a://go01-demo/warehouse/tablespace/manage...
16                                    TBLPROPERTIES (
17    'kudu.cluster_id'='f3b

In [24]:
print("Table Folder Location in Cloud Storage: ")
dataframe.loc[15][0]

Table Folder Location in Cloud Storage: 


"  's3a://go01-demo/warehouse/tablespace/managed/hive/customers'"

#### Partitions

By default, a simple HQL query scans the whole table. This slows down the performance when querying a big table. This issue could be resolved by creating partitions, which are very similar to what's in the RDBMS. In Hive, each partition corresponds to a predefined partition column(s), which maps to subdirectories in the table's directory in HDFS. When the table gets queried, only the required partitions (directory) of data in the table are being read, so the I/O and time of the query is greatly reduced. Using partition is a very easy and effective way to improve performance in Hive.

The following is an example of partition creation in HQL:

You can only change the partition column data type. We cannot add/remove a column from partition columns. If we have to change the partition design, we must back up and recreate the table, and then migrate the data. In addition, we are NOT able to change a non-partition table to a partition table directly.